In [1]:
pip install helium


  Created wheel for helium: filename=helium-3.0.5-py3-none-any.whl size=18689713 sha256=ed9967d49bf884a5ec0d6a06299c8a0174c8e70a800d9e95825e5b1e012aa372
  Stored in directory: c:\users\mahim\appdata\local\pip\cache\wheels\f4\a8\08\9f49d4fbc8781222738f824665ef6f6ac32ae00681e59e7491
Successfully built helium
Note: you may need to restart the kernel to use updated packages.


In [2]:
from helium import *
import pandas as pd
from time import sleep

In [3]:
ID_table = pd.read_excel("rxn_testing.xlsx")
ID_table

,Reaction
0,CSm
1,ACONTm
2,ICDHxm


In [4]:
## Get all the rxn full names

In [8]:
list_of_names = []
list_of_urls = []

n = len(ID_table.Reaction)

for i in range(n):
    names = ID_table.loc[i, 'Reaction']
    url = 'http://bigg.ucsd.edu/universal/reactions/'+ str(names)
    
    
    list_of_names =  list_of_names+[str(names)]
    list_of_urls = list_of_urls + [url]  
list_of_names

['CSm', 'ACONTm', 'ICDHxm']

In [9]:
len(list_of_names) == len(ID_table.Reaction) == len(list_of_urls)

True

In [11]:
list_of_ECnumber = []
amount_of_ECcount=[]
list_of_noECnumber=[]
No_BIGG_rxn=[]
rxn_name_indexing =[]
full_name = []

n=len(list_of_names)
for i in range(n): 
    url=list_of_urls[i]
    browser = start_chrome(url,headless=True)
    rxn_name=list_of_names[i]
#click on the name
    sleep(3)
    
    if Text(rxn_name).exists():
        NAME = Text(below= 'Descriptive name:').value
        full_name = full_name + [NAME]
        
        sleep(2)
#check if have any EC numbers:
        if Text("EC Number:").exists():

            ECnumber = Text(to_right_of="EC Number:").value
            list_of_ECnumber = list_of_ECnumber+[ECnumber]
            rxn_name_indexing = rxn_name_indexing + [rxn_name]

            

    else:
        print("no BIGG data found: ",  rxn_name)
        No_BIGG_rxn=No_BIGG_rxn+[rxn_name]
        list_of_noECnumber = list_of_noECnumber+[rxn_name]
    
    sleep(2)
    kill_browser()
    
print('Done!')


Done!


In [12]:
print(full_name)

['Citrate synthase', 'Aconitate hydratase', 'Isocitrate dehydrogenase NAD']


In [13]:
print(list_of_ECnumber)

['2.3.3.1', '4.2.1.3', '1.1.1.286']


### eQuilibrator

In [14]:
#Testing

#Searching by EC #, double-checking with reaction name
url ='http://equilibrator.weizmann.ac.il/enzyme?ec='+list_of_ECnumber[0]
browser = start_chrome(url)

if Text(full_name[0]).exists():
    rxn_eqn = Text(below= 'Reactions').value
    sleep (1)
    click(rxn_eqn)
    print(full_name[0],' reaction:', rxn_eqn)
    sleep(1.5)
    
 ## Adjust ionic strength, pH, pMg if needed
        #Change the pH
    write('7.2', into = S('#phField'))
        #Change the pMG
    write('3.0', into = S('#pmgField'))
        # Change the Ionic Strength
    write('0.05', into = S('#ionStrengthField'))
    #Submbit Changes
    sleep(2)

    click('UPDATE')
    ## extract standard Gibbs Free Energy & the uncertainty
 



Citrate synthase  reaction: CoA + Citrate => H2O + Acetyl-CoA + Oxaloacetate


In [15]:
kill_browser()

### Beautiful Soup

In [19]:
#install beautiful soup
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [20]:
#import necessary packages
from bs4 import BeautifulSoup
import requests

In [22]:
#define url (url is copy pasted directly - not related to helium at all)
url = "http://equilibrator.weizmann.ac.il/reaction?reactantsId=5&reactantsCoeff=1.0&reactantsName=H2O&reactantsId=15&reactantsCoeff=-1.0&reactantsName=CoA&reactantsId=23&reactantsCoeff=1.0&reactantsName=Acetyl-CoA&reactantsId=48&reactantsCoeff=1.0&reactantsName=Oxaloacetate&reactantsId=123&reactantsCoeff=-1.0&reactantsName=Citrate"

#make a GET request to get the raw HTML content
html_content = requests.get(url).text

In [23]:
#parse the html content to create a parse Tree object (soup)
#soup --> BeautifulSoup object
soup = BeautifulSoup(html_content, "lxml")

#print the parsed data of html
print(soup.prettify())

<html>
 <head>
  <title>
   Reaction
  </title>
  <link href="/static/images/equilibrator_favicon.png" rel="icon" type="image/x-icon"/>
  <link href="//cdn.muicss.com/mui-0.9.39/css/mui.min.css" rel="stylesheet" type="text/css"/>
  <link href="/static/main.css" rel="stylesheet" type="text/css"/>
  <link href="/static/style.css" rel="stylesheet" type="text/css"/>
  <link href="/static/jquery-ui.structure.min.css" rel="stylesheet" type="text/css"/>
  <link href="/static/jquery-ui.theme.min.css" rel="stylesheet" type="text/css"/>
  <script src="//cdn.muicss.com/mui-0.9.39/js/mui.min.js">
  </script>
  <script src="/static/jquery-1.11.3.min.js" type="text/javascript">
  </script>
  <script src="/static/jquery-ui.min.js" type="text/javascript">
  </script>
  <script src="/static/jquery.autocomplete.min.js" type="text/javascript">
  </script>
  <script src="/static/main.js" type="text/javascript">
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||func

In [105]:
#find all tr tags
tr = soup.find_all('tr')

#find desired tr tag
tr_tag = tr[1]

#find desired td tag
td_tag = list(tr_tag.children)[3]

#find text with Keq in it
Keq = td_tag.get_text().strip('\n')

#identify "K" so can print from Keq onwards
K_ind = Keq.find('K')
Keq = print(Keq[K_ind:])


K'eq = 1.6 × 10-7


##### EXTRA:

In [ ]:
n=len(list_of_names)
for i in range(n): 
    eQ_url = 'http://equilibrator.weizmann.ac.il' 
    browser = start_chrome(url,headless=True)
    write(full_name[i], into=text)


In [82]:
keq2 = Text(below=keq).value
print(keq2)

In [48]:
eQ_url = 'http://equilibrator.weizmann.ac.il'  
browser = start_chrome(eQ_url)
write(full_name[0], into= 'Type a single compound/enzyme name, or a full reaction (using "=" to separate the two sides)')
press(ENTER)

